In [335]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import json
import re
import ast
import itertools

In [336]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [337]:
## read_csv has the problem of reading lists as string

In [338]:
mammals_3d_den = pd.read_pickle('../data/mammals_3d_dendrites.pkl')
mammals_3d_completeden = pd.read_pickle('../data/mammals_3d_dendrites_complete.pkl')
mammals_3d_moderateden = pd.read_pickle('../data/mammals_3d_dendrites_moderate.pkl')

In [339]:
print(len(mammals_3d_den), len(mammals_3d_completeden), len(mammals_3d_moderateden))

30255 14001 16254


In [340]:
# drop the primates from completeden
primates = ['monkey', 'human', 'chimpanzee']
rodents = ['mouse', 'rat']
rodents_3d_den = mammals_3d_den[mammals_3d_den['species'].isin(rodents)]
rodents_3d_completeden = mammals_3d_completeden[mammals_3d_completeden['species'].isin(rodents)]
rodents_3d_moderateden = mammals_3d_moderateden[mammals_3d_moderateden['species'].isin(rodents)]
primates_3d_moderateden = mammals_3d_moderateden[mammals_3d_moderateden['species'].isin(primates)]
print(len(rodents_3d_den), len(rodents_3d_completeden), len(rodents_3d_moderateden), len(primates_3d_moderateden))

26881 13977 12904 3350


In [341]:
primates_3d_moderateden.species.value_counts()

human         2046
monkey         700
chimpanzee     604
Name: species, dtype: int64

In [342]:
primates_3d_moderateden.archive.value_counts()
# Jacobs human neurons usually only have basal dendrites. not good

Jacobs             2058
Bianchi             518
Briggs              334
Fujita              225
Wearne_Hof           79
Allman               64
Luebke               44
Gonzalez-Burgos      10
Zaitsev               6
Segev                 6
Szegedi_Lamsa         4
Hannibal              2
Name: archive, dtype: int64

# only use rodent neurons

In [343]:
rodents_3d_den.species.value_counts()

rat      15290
mouse    11591
Name: species, dtype: int64

In [344]:
rodents_3d_completeden.species.value_counts()

rat      9868
mouse    4109
Name: species, dtype: int64

In [345]:
rodents_3d_moderateden.species.value_counts()

mouse    7482
rat      5422
Name: species, dtype: int64

In [346]:
rodents_3d_den

,age_classification,archive,attributes,brain_region,cell_type,deposition_date,domain,experiment_condition,magnification,max_age,max_weight,min_age,min_weight,neuron_id,note,objective_type,original_format,physical_Integrity,protocol,reconstruction_software,reference_pmid,scientific_name,shrinkage_corrected,shrinkage_reported,slicing_direction,slicing_thickness,soma_surface,species,stain,strain,upload_date,3D,angles,bif_ampl_local,branch_Order,contraction,depth,diameter,eucDistance,fractal_Dim,fragmentation,height,length,n_bifs,n_branch,n_stems,neuron_name,partition_asymmetry,pathDistance,pk_classic,soma_Surface,surface,volume,width
80,young,Turner,"Diameter, 3D, Angles","[hippocampus, CA1]","[pyramidal, principal cell]",2005-12-31,"Dendrites, Soma, No Axon",Control,100,8.0,350.0,2.0,200.0,81,"When originally released, this reconstruction ...",oil,CVAPP.swc,Dendrites Complete,in vivo,Neurolucida,[9492204],rattus norvegicus,Y,Reported,coronal,80.00,672.165,rat,biocytin,Fischer 344,2006-08-01,True,True,74.6228,18.0,0.618158,395.19,0.531806,669.544,1.07516,3535.0,781.128,21804.300,81.0,168.0,6.0,n400,0.541107,1896.240,1.67043,672.16500,36679.000,6596.180,298.9460
81,young,Turner,"Diameter, 3D, Angles","[hippocampus, CA1]","[pyramidal, principal cell]",2005-12-31,"Dendrites, Soma, No Axon",Control,100,8.0,350.0,2.0,200.0,82,"When originally released, this reconstruction ...",oil,CVAPP.swc,Dendrites Complete,in vivo,Neurolucida,[9492204],rattus norvegicus,Y,Reported,coronal,80.00,903.251,rat,biocytin,Fischer 344,2006-08-01,True,True,89.5897,25.0,0.696795,364.21,1.155640,668.556,1.06807,5460.0,717.176,22216.300,113.0,233.0,7.0,n401,0.555624,1893.370,1.78362,903.25100,84796.100,30674.300,363.6990
82,young,Turner,"Diameter, 3D, Angles","[hippocampus, CA1]","[pyramidal, principal cell]",2005-12-31,"Dendrites, Soma, No Axon",Control,100,8.0,350.0,2.0,200.0,83,"When originally released, this reconstruction ...",oil,CVAPP.swc,Dendrites Complete,in vivo,Neurolucida,[9492204],rattus norvegicus,Y,Reported,coronal,80.00,845.184,rat,biocytin,Fischer 344,2006-08-01,True,True,75.6994,26.0,0.729870,313.74,0.278764,732.304,1.06048,4632.0,831.900,16317.100,89.0,182.0,4.0,n402,0.495147,1316.660,1.42035,845.18400,14822.400,2966.560,241.4460
83,young,Turner,"Diameter, 3D, Angles","[hippocampus, CA1]","[pyramidal, principal cell]",2005-12-31,"Dendrites, Soma, No Axon",Control,100,8.0,350.0,2.0,200.0,84,"When originally released, this reconstruction ...",oil,CVAPP.swc,Dendrites Complete,in vivo,Neurolucida,[9492204],rattus norvegicus,Y,Reported,coronal,80.00,528.414,rat,biocytin,Fischer 344,2006-08-01,True,True,90.4303,30.0,0.762996,255.10,0.257058,663.666,1.05999,4766.0,723.540,17697.800,100.0,206.0,6.0,n403,0.537944,1007.520,1.88745,528.41400,15463.600,2049.070,235.7200
84,young,Turner,"Diameter, 3D, Angles","[hippocampus, CA1]","[pyramidal, principal cell]",2005-12-31,"Dendrites, Soma, No Axon",Control,100,8.0,350.0,2.0,200.0,85,"When originally released, this reconstruction ...",oil,CVAPP.swc,Dendrites Complete,in vivo,Neurolucida,[9492204],rattus norvegicus,Y,Reported,coronal,80.00,638.701,rat,biocytin,Fischer 344,2006-08-01,True,True,87.4279,31.0,0.738674,227.67,0.327931,668.452,1.05664,5032.0,712.534,17344.400,105.0,213.0,3.0,n404,0.542943,1344.560,1.93384,638.70100,19024.100,3402.190,270.3660
85,young,Turner,"Diameter, 3D, Angles","[hippocampus, CA1]","[pyramidal, principal cell]",2005-12-31,"Dendrites, Soma, No Axon",Control,100,8.0,350.0,2.0,200.0,86,"When originally released, this reconstruction ...",oil,CVAPP.swc,Dendrites Complete,in vivo,Neurolucida,[9492204],rattus norvegicus,Y,Reported,coronal,80.00,985.102,rat,biocytin,Fischer 344,2006-08-01,True,True,80.7548,31.0,0.719511,193.59,0.486155,728.603,1.06799,4350.0,749.115,14724.800,82.0,169.0,5.0,n405,0.601319,1143.970,1.53977,985.10200,23513.500,5563.750,278.3180
86,young,Turner,"Diameter, 3D, Angles","[hippocampus, CA1]","[pyramidal, principal cell]",2005-12-31,"Dendrites, Soma, No Axon",C

In [347]:
rodents_3d_den.stain.value_counts().head(15)

enhanced green fluorescent protein                            5758
biocytin                                                      5005
Golgi                                                         3423
Golgi-Cox                                                     2635
NADPH-diaphorase histochesmistry                              1996
lucifer yellow                                                1656
immunostaining                                                1448
osmium and lead citrate                                       1163
green fluorescent protein                                      906
neurobiotin                                                    665
Multiple                                                       290
Not reported                                                   248
green fluorescent protein, Alexa Fluor 488, immunostaining     240
Alexa Fluor 594                                                187
neurobiotin, Alexa Fluor 594                                  

# Extract brain regions and layers

In [348]:
rodents_3d_den.brain_region.apply(tuple).value_counts()[:100]

(neocortex, occipital)                                                                  2567
(retina,)                                                                               1566
(neocortex, somatosensory, primary somatosensory, layer 5-6)                            1224
(neocortex, occipital, primary visual, superficial)                                     1111
(neocortex, occipital, primary visual, deep)                                            1078
(hippocampus, dentate gyrus, granule layer)                                             1042
(retina, ganglion layer)                                                                 859
(neocortex, occipital, primary visual)                                                   832
(neocortex, somatosensory, layer 4)                                                      523
(hippocampus, CA1)                                                                       521
(neocortex, somatosensory, primary somatosensory, layer 2-3)          

# put subiculum under hippocampus

In [349]:
rodents_3d_den.brain_region[['subiculum' in x for x in rodents_3d_den.brain_region]] = \
    [['hippocampus'] + x for x in rodents_3d_den.brain_region if 'subiculum' in x ]

/home/yueqi/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/yueqi/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/yueqi/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user

In [350]:
rodents_3d_den.brain_region[['subiculum' in x for x in rodents_3d_den.brain_region]].head()

769                               [hippocampus, subiculum]
1033                              [hippocampus, subiculum]
12261    [hippocampus, subiculum, stratum pyramidale, v...
12262    [hippocampus, subiculum, stratum pyramidale, v...
12263    [hippocampus, subiculum, stratum pyramidale, v...
Name: brain_region, dtype: object

In [351]:
rodents_3d_den.brain_region[['peripheral nervous system' in x for x in rodents_3d_den.brain_region]] = \
rodents_3d_den.brain_region[['peripheral nervous system' in x for x in rodents_3d_den.brain_region]].apply( \
        lambda x: ["spinal cord"] + [y for y in x if y !='peripheral nervous system'])

/home/yueqi/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/yueqi/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/yueqi/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user

In [352]:
rodents_3d_den.brain_region[['peripheral nervous system' in x for x in rodents_3d_den.brain_region]]

Series([], Name: brain_region, dtype: object)

In [353]:
def process_brain_region(row):
    br = row['brain_region'].copy()
    layer_list = list(filter(lambda x: x.startswith('layer'), br))
    rough_layer_list = list(filter(lambda x: x=='superficial' or x=='deep', br))
    if len(layer_list) > 1:
        print('multiple layer information: %s' % str(layer_list))
    if len(layer_list) >= 1:
        layer = layer_list[0]
        for l in layer_list:
            br.remove(l)
    else:
        layer = None
    
    if len(rough_layer_list) >= 1:
        rough_layer = rough_layer_list[0]
        for l in rough_layer_list:
            br.remove(l)
    else:
        rough_layer = None
        
    hemi_list = list(filter(lambda x: x=='left' or x=='right', br))
    if len(hemi_list) >=1:
        hemi = hemi_list[0]
        br.remove(hemi)
    else:
        hemi = None
    
    
    if len(br) > 0:
        region_1 = br[0]
        br.remove(region_1)
    else:
        region_1 = None
    
    if len(br) > 0:
        region_2 = br[0]
        br.remove(region_2)
    else:
        region_2 = None
    if len(br) > 0:
        region_3 = br[0]
        br.remove(region_3)
    else:
        region_3 = None
    if len(br) > 0:
        region_4 = br
    else:
        region_4 = None
    return pd.Series({'neuron_id': row['neuron_id'], 'raw_layer': layer, 'ds_layer': rough_layer, 
                      'hemisphere': hemi, 
                     'br1': region_1, 'br2': region_2, 'br3': region_3, 'br4': region_4})

In [354]:
rodents_3d_den_region = rodents_3d_den.merge(rodents_3d_den.apply(process_brain_region, axis=1), on=['neuron_id'])

multiple layer information: ['layer 5', 'layer 5a']
multiple layer information: ['layer 5', 'layer 5a']
multiple layer information: ['layer 5', 'layer 5a']
multiple layer information: ['layer 5', 'layer 5a']
multiple layer information: ['layer 5', 'layer 5a']
multiple layer information: ['layer 5', 'layer 5a']
multiple layer information: ['layer 5', 'layer 5a']
multiple layer information: ['layer 5', 'layer 5a']
multiple layer information: ['layer 5', 'layer 5a']
multiple layer information: ['layer 5', 'layer 5a']
multiple layer information: ['layer 5', 'layer 5b']
multiple layer information: ['layer 5', 'layer 5b']
multiple layer information: ['layer 5', 'layer 5b']
multiple layer information: ['layer 5', 'layer 5b']
multiple layer information: ['layer 5', 'layer 5b']
multiple layer information: ['layer 5', 'layer 5b']
multiple layer information: ['layer 5', 'layer 5b']
multiple layer information: ['layer 5', 'layer 5b']
multiple layer information: ['layer 5', 'layer 5b']
multiple lay

# simplify cortical layers

In [355]:
rodents_3d_den_region.raw_layer.value_counts()

layer 2-3         2341
layer 5           1979
layer 5-6         1298
layer 3           1082
layer 4            994
layer 6            482
layer 1            289
layer 5b           160
layer 2            120
layer 3-4           97
layer 6b            95
layer 6a, left      80
layer 5a            30
Name: raw_layer, dtype: int64

In [356]:
raw_layer_convert_exact = {'layer 2-3': 'layer 2-3', 
                     'layer 5': 'layer 5', 
                     'layer 5-6': 'layer 5-6', 
                     'layer 3': 'layer 3', 
                     'layer 4': 'layer 4', 
                     'layer 6': 'layer 6',
                     'layer 1': 'layer 1', 
                     'layer 5b': 'layer 5', 
                     'layer 2': 'layer 2', 
                     'layer 3-4': 'layer 2-3', # the paper with these neurons actually say layer 2/3
                     'layer 6b': 'layer 6',
                     'layer 6a, left': 'layer 6', 
                     'layer 5a': 'layer 5'}
raw_layer_convert_rough = {'layer 2-3': 'layer 2-3', 
                     'layer 5': 'layer 5-6', 
                     'layer 5-6': 'layer 5-6', 
                     'layer 3': 'layer 2-3', 
                     'layer 4': 'layer 4', 
                     'layer 6': 'layer 5-6',
                     'layer 1': 'layer 1', 
                     'layer 5b': 'layer 5-6', 
                     'layer 2': 'layer 2-3', 
                     'layer 3-4': 'layer 2-3', # the paper with these neurons actually say layer 2/3
                     'layer 6b': 'layer 5-6',
                     'layer 6a, left': 'layer 5-6', 
                     'layer 5a': 'layer 5-6'}
raw_layer_convert_deep_sup = {'layer 2-3': 'superficial', 
                     'layer 5': 'deep', 
                     'layer 5-6': 'deep', 
                     'layer 3': 'superficial', 
                     'layer 4': 'deep', 
                     'layer 6': 'deep',
                     'layer 1': 'superficial', 
                     'layer 5b': 'deep', 
                     'layer 2': 'superficial', 
                     'layer 3-4': 'superficial', # the paper with these neurons actually say layer 2/3
                     'layer 6b': 'deep',
                     'layer 6a, left': 'deep', 
                     'layer 5a': 'deep'}

In [357]:
rodents_3d_den_region['raw_layer'].map(raw_layer_convert_deep_sup)

0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
5                NaN
6                NaN
7                NaN
8                NaN
9                NaN
10               NaN
11               NaN
12               NaN
13               NaN
14               NaN
15               NaN
16               NaN
17               NaN
18               NaN
19               NaN
20               NaN
21               NaN
22               NaN
23               NaN
24               NaN
25               NaN
26               NaN
27               NaN
28               NaN
29               NaN
30               NaN
31               NaN
32               NaN
33               NaN
34               NaN
35               NaN
36               NaN
37               NaN
38               NaN
39               NaN
40               NaN
41               NaN
42               NaN
43               NaN
44               NaN
45               NaN
46               NaN
47           

In [358]:
rodents_3d_den_region['exact_layer'] = rodents_3d_den_region['raw_layer'].map(raw_layer_convert_exact)
rodents_3d_den_region['rough_layer'] = rodents_3d_den_region['raw_layer'].map(raw_layer_convert_rough)
new_ds_layer = rodents_3d_den_region['raw_layer'].map(raw_layer_convert_deep_sup)

In [359]:
rodents_3d_den_region['ds_layer'].value_counts()

superficial    1261
deep           1091
Name: ds_layer, dtype: int64

In [360]:
rodents_3d_den_region['ds_layer'][~pd.isnull(new_ds_layer)] = new_ds_layer[~pd.isnull(new_ds_layer)]

/home/yueqi/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [361]:
rodents_3d_den_region['ds_layer'].value_counts()

deep           6209
superficial    5190
Name: ds_layer, dtype: int64

In [362]:
rodents_3d_den_region['exact_layer'].value_counts()

layer 2-3    2438
layer 5      2169
layer 5-6    1298
layer 3      1082
layer 4       994
layer 6       657
layer 1       289
layer 2       120
Name: exact_layer, dtype: int64

In [363]:
rodents_3d_den_region['rough_layer'].value_counts()

layer 5-6    4124
layer 2-3    3640
layer 4       994
layer 1       289
Name: rough_layer, dtype: int64

# Filter brain regions

In [364]:
rodents_3d_den_region.br1.value_counts()

neocortex                     16473
hippocampus                    3644
retina                         2481
basal ganglia                  1499
spinal cord                     905
main olfactory bulb             536
amygdala                        418
ventral striatum                295
brainstem                       186
cerebellum                      148
myelencephalon                   86
hypothalamus                     79
entorhinal cortex                57
ventral thalamus                 27
anterior olfactory nucleus       27
basal forebrain                   6
dorsal thalamus                   6
mesencephalon                     3
thalamus                          3
pons                              1
accessory olfactory bulb          1
Name: br1, dtype: int64

In [365]:
# drop entorhinal cortex, myelencephalon. only keep the ones > 100

In [366]:
br1_filter = rodents_3d_den_region.br1.value_counts().iloc[:10].index
br1_filter

Index(['neocortex', 'hippocampus', 'retina', 'basal ganglia', 'spinal cord',
       'main olfactory bulb', 'amygdala', 'ventral striatum', 'brainstem',
       'cerebellum'],
      dtype='object')

In [367]:
rodents_3d_den_region = rodents_3d_den_region[rodents_3d_den_region['br1'].isin(list(br1_filter))]
len(rodents_3d_den_region)

26585

In [368]:
rodents_3d_den_region.br1.value_counts()

neocortex              16473
hippocampus             3644
retina                  2481
basal ganglia           1499
spinal cord              905
main olfactory bulb      536
amygdala                 418
ventral striatum         295
brainstem                186
cerebellum               148
Name: br1, dtype: int64

In [369]:
pd.crosstab(rodents_3d_den_region.br1, rodents_3d_den_region.br2)

br2,CA1,CA2,CA3,L2,L5,L5-L6,L5-S1,Multiple,anterior cingulate,basolateral amygdala complex,cerebellar cortex,deep cerebellar nuclei,dentate gyrus,dorsal root ganglion,dorsomedial,external plexiform layer,facial nucleus,frontal,ganglion layer,globus pallidus,glomerular layer,hypoglossal nucleus,inner plexiform layer,internal plexiform layer,lumbar,main intercalated nucleus,medial prefrontal,mitral layer,nucleus accumbens,occipital,pedunculopontine nucleus,perirhinal,prefrontal,presubiculum,primary auditory,reticular formation,somatosensory,spinal trigeminal nucleus,stratum granulosum,striatum,subiculum,substantia nigra,temporal,ventral striatum,vermis
br1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
amygdala,0,0,0,0,0,0,0,0,0,414,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
basal ganglia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,89,0,0,0,0,0,0,0,0,163,0,0,0,0,0,0,0,0,0,0,420,0,22,0,805,0
brainstem,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,32,0,0,0,0,43,0,0,0,0,0,0,0,0,31,0,0,0,0,12,0,68,0,0,0,0,0,0,0
cerebellum,0,0,0,0,0,0,0,0,0,0,60,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
hippocampus,1133,8,290,0,0,0,0,0,0,0,0,0,1983,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,112,0,0,0,0
main olfactory bulb,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,48,0,0,4,0,0,0,20,0,0,0,0,0,0,0,0,0,0,114,0,0,0,0,0,0
neocortex,0,0,0,0,0,0,0,129,475,0,0,0,0,0,22,0,0,1061,0,0,0,0,0,0,0,0,1907,0,0,6503,0,170,512,18,2,0,5325,0,0,0,0,0,41,0,0
retina,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,859,0,0,0,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
spinal cord,0,0,0,10,14,1,17,0,0,0,0,0,0,193,0,0,0,0,0,0,0,0,0,0,670,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [370]:
rodents_3d_den_region.br2.value_counts(dropna=False)

occipital                       6503
somatosensory                   5325
NaN                             2413
dentate gyrus                   1983
medial prefrontal               1907
CA1                             1133
frontal                         1061
ganglion layer                   859
ventral striatum                 805
lumbar                           670
prefrontal                       512
anterior cingulate               475
nucleus accumbens                458
striatum                         420
basolateral amygdala complex     414
CA3                              290
dorsal root ganglion             193
perirhinal                       170
Multiple                         129
stratum granulosum               114
subiculum                        112
globus pallidus                   89
spinal trigeminal nucleus         68
cerebellar cortex                 60
inner plexiform layer             56
glomerular layer                  48
hypoglossal nucleus               43
t

# Filter cell types

In [371]:
rodents_3d_den_celltype = rodents_3d_den_region[~pd.isnull(rodents_3d_den_region['cell_type'])]
len(rodents_3d_den_celltype)

26341

In [372]:
cell_type_options_1 = ['Glia', 'interneuron', 'principal cell', 'sensory receptor', 'Not reported'] 

In [373]:
def process_cell_types(row):
    cell_type = row['cell_type'].copy()
    ct_list = list(filter(lambda x: x.lower() in [y.lower() for y in cell_type_options_1], cell_type))
    if len(ct_list) > 1:
        print('multiple cell type information: %s' % str(ct_list))
    if len(ct_list) >= 1:
        ct1 = ct_list[0]
        for l in ct_list:
            cell_type.remove(l)
    else:
        ct1 = None
    
    
    if len(cell_type) > 0:
        ct2 = cell_type
    else:
        ct2 = None

    return pd.Series({'neuron_id': row['neuron_id'], 'cell_type_1': ct1, 'cell_type_2': ct2})

In [374]:
rodents_3d_den_celltype = rodents_3d_den_celltype.merge(rodents_3d_den_celltype.apply(process_cell_types, axis=1), on=['neuron_id'])

In [375]:
rodents_3d_den_celltype[(rodents_3d_den_celltype.archive=='Allen Cell Types') & \
                       (rodents_3d_den_celltype.cell_type_1=='principal cell')].cell_type_2.astype('str').value_counts()

['Spiny', 'Homeobox Protein Cux-2-positive']                                 85
['Spiny', 'Steroid Hormone Receptor Ad4BP-positive']                         46
['Spiny', 'Connective Tissue Growth Factor positive']                        23
['Spiny', 'Plasma Retinol-Binding Protein-Positive']                         21
['Non Voltage Gated 1 Alpha Subunit-positive', 'Sodium Channel', 'Spiny']    11
['Steroid Hormone Receptor Ad4BP-positive', 'Spiny']                         10
['Spiny']                                                                     7
['Spiny', 'Sodium Channel', 'Non Voltage Gated 1 Alpha Subunit-positive']     5
['Retinoid-Related Orphan Receptor-Beta-positive', 'Spiny']                   4
['Neurotensin Receptor 1-positive', 'Spiny']                                  3
['Plasma Retinol-Binding Protein-Positive', 'Spiny']                          3
['Somatostatin (SOM)-positive', 'Spiny']                                      1
Name: cell_type_2, dtype: int64

In [376]:
rodents_3d_den_celltype.cell_type_1.value_counts(dropna=False)

principal cell      17300
interneuron          7888
Glia                  892
sensory receptor      261
Name: cell_type_1, dtype: int64

In [377]:
pd.crosstab(rodents_3d_den_celltype.br1, rodents_3d_den_celltype.cell_type_1)

cell_type_1,Glia,interneuron,principal cell,sensory receptor
br1,,,,
amygdala,0,62,356,0
basal ganglia,0,28,1471,0
brainstem,0,12,106,68
cerebellum,77,31,40,0
hippocampus,0,911,2733,0
main olfactory bulb,0,161,375,0
neocortex,149,5656,10480,0
retina,274,816,1335,0
spinal cord,392,211,109,193


In [378]:
rodents_3d_den_celltype[rodents_3d_den_celltype.cell_type_1=='Glia'].cell_type.astype('str').value_counts()

['Glia', 'microglia', 'Iba1-positive']                             392
['Glia']                                                           274
['Glia', 'microglia']                                              149
['Glia', 'Oligodendrocyte', 'Precursor cell', 'NG2-expressing']     77
Name: cell_type, dtype: int64

In [379]:
rodents_3d_den_celltype[rodents_3d_den_celltype.cell_type_1=='sensory receptor'].cell_type.astype('str').value_counts()

['sensory receptor']                                                    193
['Sp5c-Pr5 ascending', 'Intersubnuclear neuron', 'sensory receptor']     68
Name: cell_type, dtype: int64

## delete Glia and sensory receptors

In [380]:
rodents_3d_den_celltype = rodents_3d_den_celltype[~rodents_3d_den_celltype.cell_type_1.isin(['sensory receptor', 'Glia'])]
print(26341-892-261)
len(rodents_3d_den_celltype)

25188


25188

In [381]:
rodents_3d_den_celltype.cell_type_1.value_counts(dropna=False)

principal cell    17300
interneuron        7888
Name: cell_type_1, dtype: int64

In [382]:
# these hippocampus granule cells are labeled with 'interneuron' by mistake.
rodents_3d_den_celltype.loc[rodents_3d_den_celltype.archive=='Tessarollo_Yanpallewar', 'cell_type_1'] = 'principal cell'
rodents_3d_den_celltype.loc[rodents_3d_den_celltype.archive=='Tessarollo_Yanpallewar', 'cell_type_1']

16518    principal cell
16519    principal cell
16520    principal cell
16521    principal cell
16522    principal cell
16523    principal cell
16524    principal cell
16525    principal cell
16526    principal cell
16527    principal cell
16528    principal cell
16529    principal cell
16530    principal cell
16531    principal cell
16532    principal cell
16533    principal cell
16534    principal cell
16535    principal cell
16536    principal cell
16537    principal cell
16538    principal cell
16539    principal cell
16540    principal cell
16541    principal cell
16542    principal cell
16543    principal cell
16544    principal cell
16545    principal cell
16546    principal cell
16547    principal cell
16548    principal cell
16549    principal cell
16550    principal cell
16551    principal cell
16552    principal cell
16553    principal cell
16554    principal cell
16555    principal cell
16556    principal cell
16557    principal cell
16558    principal cell
16559    princip

In [388]:
rodents_3d_den_celltype.loc[(rodents_3d_den_celltype.cell_type_1=='interneuron') & \
                           (rodents_3d_den_celltype.br1=='main olfactory bulb') & \
                           (rodents_3d_den_celltype.archive.isin(['Caille', 'Urban'])), 'cell_type_1'] = 'principal cell'

In [391]:
rodents_3d_den_celltype.loc[(rodents_3d_den_celltype.cell_type_1=='principal cell') & 
                           (rodents_3d_den_celltype.br1=='main olfactory bulb') & 
                           (rodents_3d_den_celltype.archive.isin(['Caille', 'Urban'])), 'cell_type_1']

12794    principal cell
12795    principal cell
12796    principal cell
12797    principal cell
12798    principal cell
12799    principal cell
12800    principal cell
12801    principal cell
12802    principal cell
12803    principal cell
12804    principal cell
12805    principal cell
12806    principal cell
12807    principal cell
12808    principal cell
12809    principal cell
12810    principal cell
12811    principal cell
12812    principal cell
12813    principal cell
12814    principal cell
12815    principal cell
12816    principal cell
12817    principal cell
12818    principal cell
12819    principal cell
12820    principal cell
12821    principal cell
12822    principal cell
12823    principal cell
12824    principal cell
12825    principal cell
12826    principal cell
12827    principal cell
12828    principal cell
12829    principal cell
12830    principal cell
12831    principal cell
12832    principal cell
12833    principal cell
12834    principal cell
12835    princip

# pincipal cells -- simplify subtypes

In [392]:
principal_cells = rodents_3d_den_celltype[rodents_3d_den_celltype.cell_type_1=='principal cell']
principal_cells.cell_type_2.astype('str').value_counts()

['pyramidal']                                                                10408
['granule']                                                                   1106
['medium spiny']                                                              1008
['ganglion']                                                                   703
None                                                                           506
['granule', 'adult-born']                                                      465
['medium spiny', 'projection']                                                 440
['adult-born', 'granule']                                                      194
['pyramidal', 'glutamatergic']                                                 168
['newborn', 'adult-born', 'granule']                                           154
['Large', 'Aspiny']                                                            146
['granule', 'adult-born', 'newborn', 'Doublecortin (DCX)-containing']          141
['Mo

In [393]:
#principal_cells.cell_type_2.astype('str').value_counts().to_csv('../data/principal_cells_ct2.csv')
# then add curated cell_type_2 label

In [394]:
principal_ct2_convert = pd.read_csv('../data/principal_cells_ct2.csv', index_col=None)
principal_ct2_convert.head()

,cell_type_2,count,cell_type_2_convert
0,['pyramidal'],10431,pyramidal
1,['medium spiny'],1008,medium spiny
2,['ganglion'],703,ganglion
3,['granule'],621,granule
4,None,545,None


In [395]:
principal_ct2_mapping = dict()
for x in principal_ct2_convert.iterrows():
    principal_ct2_mapping[x[1].cell_type_2] = x[1].cell_type_2_convert
principal_ct2_mapping

{'None': 'None',
 "['A1 type', 'ON type', 'ganglion']": 'ganglion',
 "['A2 inner type', 'ON type', 'ganglion']": 'ganglion',
 "['A2 outer type', 'OFF type', 'ganglion']": 'ganglion',
 "['Alpha', 'Gastrocnemius', 'Motoneuron']": 'Motoneuron',
 "['B1 type', 'OFF type', 'ganglion']": 'ganglion',
 "['B3 outer type', 'OFF type', 'ganglion']": 'ganglion',
 "['C2 inner type', 'ON type', 'ganglion']": 'ganglion',
 "['C2 outer type', 'OFF type', 'ganglion']": 'ganglion',
 "['Cajal-Retzius', 'glutamatergic']": 'Cajal-Retzius',
 "['Cajal-Retzius']": 'Cajal-Retzius',
 "['Calbindin (CB)-positive', 'pyramidal']": 'pyramidal',
 "['Dbx1-negative', 'Non-glutamatergic']": nan,
 "['Doublecortin (DCX)-containing', 'Immature', 'Neuroblast']": 'Neuroblast',
 "['Excitatory', 'early-born']": nan,
 "['Excitatory', 'translocating']": nan,
 "['External tufted cell (ETC)']": nan,
 "['Fast-spiking', 'glutamatergic']": nan,
 "['Flattened', 'Waldeyer', 'anterolateral-targeting', 'projection']": nan,
 "['GABAergic', 

In [396]:
principal_cells['cell_type_2_curated'] = principal_cells['cell_type_2'].astype('str').map(principal_ct2_mapping)

/home/yueqi/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [397]:
principal_cells['cell_type_2_curated'].value_counts()

pyramidal          11389
granule             2163
medium spiny        1567
ganglion            1333
None                 506
Aspiny               146
Motoneuron           136
Cajal-Retzius         71
mitral                52
projection            48
glutamatergic         43
stellate              40
Purkinje              36
tufted                33
Non-cholinergic       26
Neuroblast            25
dopaminergic          22
Name: cell_type_2_curated, dtype: int64

# interneurons --simplify subtypes

In [398]:
interneurons = rodents_3d_den_celltype[rodents_3d_den_celltype.cell_type_1=='interneuron']
interneurons.cell_type_2.astype('str').value_counts()

['Nitrergic']                                                                                                                                                           1996
None                                                                                                                                                                    1221
['GABAergic']                                                                                                                                                            701
['bipolar', 'cone']                                                                                                                                                      305
['Martinotti']                                                                                                                                                           165
['Large', 'basket']                                                                                                                    

In [399]:
len(interneurons)

7294

In [400]:
#interneurons.cell_type_2.astype('str').value_counts().to_csv('../data/interneurons_ct2.csv')
# then add curated cell_type_2 label

In [401]:
interneuron_ct2_convert = pd.read_csv('../data/interneurons_ct2.csv', index_col=None)
interneuron_ct2_convert.head()

,cell_type_2,count,cell_type_2_convert
0,['Nitrergic'],1996,Nitrergic
1,None,1221,None
2,['GABAergic'],701,GABAergic
3,['granule'],485,granule
4,"['bipolar', 'cone']",305,bipolar


In [402]:
interneuron_ct2_mapping = dict()
for x in interneuron_ct2_convert.iterrows():
    interneuron_ct2_mapping[x[1].cell_type_2] = x[1].cell_type_2_convert
interneuron_ct2_mapping

{'None': 'None',
 "['Alpha-actinin 2 (AAc)-positive', 'neurogliaform']": 'neurogliaform',
 "['Aspiny', 'Parvalbumin (PV)-positive']": 'Aspiny',
 "['Aspiny', 'Plasma Retinol-Binding Protein-Positive']": nan,
 "['Aspiny', 'Serotonin receptor type 3A(5HT3)-positive']": 'Aspiny',
 "['Aspiny', 'Somatostatin (SOM)-positive']": 'Aspiny',
 "['Aspiny', 'Steroid Hormone Receptor Ad4BP-positive']": nan,
 "['Aspiny', 'serotonergic']": 'Aspiny',
 "['Aspiny']": nan,
 "['Back-Projecting', 'GABAergic', 'Somatostatin (SOM)-positive', 'mGluR1alpha-positive']": nan,
 "['Cajal-Retzius']": nan,
 "['Calbindin (CB)-negative', 'Calretinin (CR)-negative', 'glycinergic', 'pre-motor']": nan,
 "['Calbindin (CB)-negative', 'Calretinin (CR)-positive', 'glycinergic', 'pre-motor']": nan,
 "['Calbindin (CB)-negative', 'Parvalbumin (PV)-negative', 'glycinergic', 'pre-motor']": nan,
 "['Calbindin (CB)-negative', 'Somatostatin (SOM)-positive', 'oriens-lacunosum moleculare (OLM)', 'GABAergic', 'mGluR1alpha-positive', 'Par

In [403]:
interneurons['cell_type_2_curated'] = interneurons['cell_type_2'].astype('str').map(interneuron_ct2_mapping)
interneurons['cell_type_2_curated'].head()

/home/yueqi/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


62    NaN
63    NaN
64    NaN
65    NaN
66    NaN
Name: cell_type_2_curated, dtype: object

In [404]:
interneurons['cell_type_2_curated'].value_counts()

Nitrergic         1996
None              1221
GABAergic          701
basket             608
bipolar            478
amacrine           369
Martinotti         282
neurogliaform      168
Aspiny             142
bitufted            88
Chandelier          61
pyramidal           61
stellate            58
double bouquet      57
granule              1
Name: cell_type_2_curated, dtype: int64

# Save data for now

In [405]:
principal_cells.to_pickle('../data/rodent_3d_dendrites_br-ct-filter-1_principal.pkl')

In [406]:
interneurons.to_pickle('../data/rodent_3d_dendrites_br-ct-filter-1_interneuron.pkl')